## Averaging last 4 attention distributions
We average together the last 4 attention distributions of the input. We experiment with using different number of attention heads.

* IDEA 1: The current issue with this code is that we are still require to find the location of the biased word and we extract the attention distribution based on that particular index of the biased word --> Instead what we should do is feed in the entire distribution into some sort of an LSTM structure and then use that directly to predict the attention. 

In [30]:
import sys; sys.path.append("../../..")
import torch 
from src.experiment import AttentionExperiment, ClassificationExperiment
from src.dataset import ExperimentDataset
from src.params import Params

In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
params = Params.read_params("experiment_params.json")

In [33]:
# Loading in the dataset that we are using in this experiments 
# typically this dataset is the small set of ground-truth labels
dataset = ExperimentDataset.init_dataset(params)

02/17/2020 15:53:50 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
386it [00:00, 5142.02it/s]


In [34]:
attention_dataloader = dataset.return_dataloader()

Attention Experiment: 
* Is a class that wraps useful methods to extract attention distributions from a given BERT-based model 
* In the config file the user needs to specify a .ckpt file for a trained BERT-based model from which 
     we want to extract attention scores
* The user needs to instantiate the attention experiment with a function that tells the model how to run 
 inference on the given model 

In [35]:
attention_experiment = AttentionExperiment.initialize_attention_experiment(params.intermediary_task, verbose=True)

02/17/2020 15:53:53 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


The len of our vocabulary is 30523
Cuda is set to true


02/17/2020 15:53:53 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
02/17/2020 15:53:53 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpgm0a77kv
02/17/2020 15:53:57 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

02/17/2020 15:54:00 - INFO - pytor

Succesfully loaded in attention experiment!


In [37]:
dataset

Length: 324 Keys: dict_keys(['pre_ids', 'masks', 'pre_lens', 'post_in_ids', 'post_out_ids', 'pre_tok_label_ids', 'post_tok_label_ids', 'rel_ids', 'pos_ids', 'categories', 'index', 'bias_label'])

extract_attention_scores() works out of the box because the attention experiment has the config file saved, and knows what BERT model to use/load in, which layers to extract the attention scores from, and what the inference function is that should be used on this particular BERT model.

Attention_scores is then a list of dictionaries. The keys in this dictionary are the specific layers of a BERT model and the values are the corresponding attention distributions extracted from that particular layer.

In [38]:
attention_scores = attention_experiment.extract_attention_scores(attention_dataloader)

We extract the index of the first predicted biased word. 

In [43]:
from src.utils.data_utils import get_bias_indices

intermediary_labels = dataset.get_val('pre_tok_label_ids')
bias_indices = get_bias_indices(intermediary_labels) 

#### NOTE: This is the main problem currently we are still windowing the attention distribution around the predicted biased word

In [ ]:
bias_indices

In [49]:
from src.utils.attention_utils import return_idx_attention_dist, sum_attention_dist, window_attention_dist

In [48]:
# Extracting attention scores of biased token
biased_tok_dist = return_idx_attention_dist(attention_scores, bias_indices)

# Summing together bias distributions
num_attention_layers = len(params.intermediary_task['attention']['layers'])
avg_dist = [dist/num_attention_layers for dist in sum_attention_dist(biased_tok_dist)]

#### Removed indices are the indices of the attention distributions that could not be windowed because they fall outside of the center.


In [50]:
windowed_data, removed_indices = window_attention_dist(avg_dist, bias_indices, window_size=params.final_task['window_size'])
dataset.remove_indices(removed_indices)
dataset.add_data(torch.stack(windowed_data), "attention_dist")
dataset.shuffle_data()

### This is where the classification experiment starts

We create a classification experiment that contains useful methods for classifying bias based on the attention distributions. 

In [54]:
classification_experiment = ClassificationExperiment.init_cls_experiment(params.final_task)

In [55]:
from src.utils.classification_utils import run_bootstrapping

In [56]:
statistics = run_bootstrapping(classification_experiment, dataset, params, input_key='attention_dist', label_key='bias_label', threshold=0.42)

0.7037181854248047
0.7043883800506592
0.6990550756454468
0.6923120617866516
0.6903248429298401
0.6894090175628662


0.6909458637237549
0.6876672506332397
0.6945733428001404
0.671580970287323
0.677747368812561
0.6704761981964111


0.6847048401832581
0.6806734204292297
0.6949846744537354
0.6590075492858887
0.6698207259178162
0.659464418888092


0.681697428226471
0.6761829257011414
0.6967228055000305
0.6474052667617798
0.6628074645996094
0.6495528221130371


0.6793630719184875
0.672515332698822
0.6989179849624634
0.6368988156318665
0.6566672325134277
0.640537679195404


...

0.609732449054718
0.640347421169281
0.6367402672767639
0.6799254417419434
0.7116172909736633
0.6709207892417908


0.6096956729888916
0.63995361328125
0.6369105577468872
0.6796503663063049
0.7117881774902344
0.6714175939559937


0.6097739934921265
0.6395638585090637
0.6370592713356018
0.6795604825019836
0.711753785610199
0.6712080836296082


0.6097559928894043
0.6398577094078064
0.6367695927619934
0.6799326539039612
0.7114516496658325
0.6704334616661072


KeyboardInterrupt: 